In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import re
import plotly as pl 
import matplotlib.pyplot as plt
from pathlib import Path

# Import db related libraries
import psycopg2
import sys
import boto3
import os
import sqlalchemy

In [15]:
# Loading csv from S3 Bucket.
lat_long_data = "https://chocolate-final-project-21.s3.us-east-2.amazonaws.com/country_and_usa_states_lat_long.csv"
lat_long_df = pd.read_csv(lat_long_data)

#file_path = Path('country_and_usa_states_lat_long.csv')
#df = pd.read_csv(file_path)
lat_long_df.head()



,country_code,latitude,longitude,country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [18]:
# Load csv of cleaned version
clean_flavors_data = "https://chocolate-final-project-21.s3.us-east-2.amazonaws.com/clean_flavors_of_cacao.csv"
clean_flavors_df = pd.read_csv(clean_flavors_data)

clean_flavors_df.head()


,Company,Bean_Origin_or_Bar_Name,REF,Review_Date,Cocoa_Percent,Company_Location,Rating,Bean_Type,Broad_Bean_Origin,Ingredients,Most_Memorable_Characteristics,country_code,latitude,longitude,continent
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe,"4- B,S,C,L","sweet, chocolatey, vegetal",ST,0.186360,6.613081,Africa
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo,"4- B,S,C,L","burnt wood, earthy, choco",TG,8.619543,0.824782,Africa
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo,"4- B,S,C,L","roasty, acidic, nutty",TG,8.619543,0.824782,Africa
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo,"4- B,S,C,L","mild profile, chocolaty, spice",TG,8.619543,0.824782,Africa
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru,"4- B,S,C,L","grainy texture, cocoa, sweet",PE,-9.189967,-75.015152,South America


In [17]:
# Store environmental variable
from getpass import getpass
enter_password = getpass('Enter database password')


connection = psycopg2.connect(
    host = "database-chocolate.cafzzay3t2tr.us-east-2.rds.amazonaws.com",
    port = 5432,
    user = 'postgres',
    password = enter_password,
    database = 'postgres'
    )

# If curser object = 0, then connection was successfully established
cursor = connection.cursor()
cursor

KeyboardInterrupt: Interrupted by user

In [ ]:
# Create location table - this table has been set up already

#cursor.execute(""" CREATE TABLE location_table(
#country_code text,
#latitude float,
#longitude float,
#country text)""")

#connection.commit()

In [ ]:
# Create clean_flavors_of_cacao table on postgres.

#country text)""")
cursor.execute(""" CREATE TABLE clean_flavors_table(
Company text,
Bean_Origin_or_Bar_Name text,
REF text,
Review_Date text,
Cocoa_Percent float,
Company_Location text,
Rating float,
Bean_Type text,
Broad_Bean_Origin text,
Ingredients text,
Most_Memorable_Characteristics text,
country_code text,
latitude float,
longitude float,
continent text
)""")




In [ ]:
# Load data into postgres table from csv currently - have not figured out how to directly open from s3.
# No need to re-run this cell as csv file has already been imported into table.

with open ('country_and_usa_states_lat_long.csv', 'r') as row:
           next(row) # skip the header row
           cursor.copy_from(row, 'location_table', sep = ',')
                   
connection.commit()

In [ ]:
# Load clean_flavors data into postgres table from csv currently - have not figured out how to directly open from s3.

with open ('clean_flavors_of_cacao.csv', 'r') as row:
           next(row) # skip the header row
           cursor.copy_from(row, 'clean_flavors_table', sep = ',')
                   
connection.commit()

In [8]:
# Query and view location_table from postgres.
sql = """
SELECT * FROM location_table
"""

location_from_db = pd.read_sql(sql, con=connection)

In [9]:
location_from_db.head(5)

,country_code,latitude,longitude,country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [ ]:
# Query and view table clean_flavours_table
sql = """
SELECT * FROM clean_flavors_table
"""

clean_flavors_from_db = pd.read_sql(sql, con=connection)

In [ ]:
clean_flavors_from_db.head()

In [11]:
# Close connection
cursor.close()

In [12]:
# Make sure connection is closed. If object returns 1, then connection is closed.

cursor

<cursor object at 0x7f8e54857550; closed: -1>